In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.8 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata


from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "online-rag-project"

# pc.create_index(
#     name=index_name,
#     dimension=768,
#     metric="cosine",
#     spec=ServerlessSpec(cloud="aws", region="us-east-1"),
# )

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vector = embeddings.embed_query("We are building a RAG Text")
vector[:5]

[0.03732863813638687,
 -0.054927337914705276,
 -0.05723658204078674,
 -0.016060668975114822,
 0.008930283598601818]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [ ]:
document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [ ]:
len(documents)

10

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['ad704d94-9f35-4b49-99a0-202433b46af2',
 'fd19518c-22c7-44c3-987f-35144bfa817e',
 'a8127df9-e015-438f-986a-e1c60b458f07',
 '1090ebd7-5ff8-4801-a042-8e2bb3f2c996',
 'a828eab1-2e77-4fcf-9120-4373f2654f4e',
 '142132f8-257b-4aed-877e-4ad37d8ce506',
 '539b0ce5-022a-4267-9e0a-5cb65761b9dd',
 '5d8a5ec4-1d42-4930-a19f-e3c4b7d5ecfb',
 'd62b757d-26fb-45d5-bf42-41380cb957fb',
 '719bf072-6bcc-47ac-8dfb-40c2d75c15dc']

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [ ]:
import langchain_google_genai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
model : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GOOGLE_API_KEY,
)

In [ ]:
def answer_to_user(query: str):
    results = vector_store.similarity_search(query, k=2)
    # Convert results to a string that can be included in the system message
    context_str = "\n".join([f"* {res.page_content} [{res.metadata}]" for res in results])

    messages = [
        (
            "system",
            f"You are a helpful assistant. Use the following context to answer the query accurately:\n{context_str}"
        ),
        (
            "context",
            results
        ),
        ("human", query),
    ]

    final_answer = model.invoke(messages)

    return final_answer;



In [ ]:
answer  = answer_to_user("LangChain provides abstractions to make working with LLM easy")
answer.content

In [ ]:
def answer_to_user(query: str):
    vector_results = vector_store.similarity_search(query, k=2)


    final_answer = model.invoke(f"Answer this user query: {query}, Here are some references to answer {vector_results}")

    return final_answer;